In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
data = pd.read_csv("myDataFile.csv", low_memory =False)

In [3]:
items_name = list(data.columns)
items_dict = dict()

for i, it in enumerate(items_name):
    items_dict[it] = i + 1

In [4]:
trans = list()

for i, r in data.iterrows():
    transaction = set()
    
    for item in items_dict:
        if r[item] == 't':
            transaction.add(items_dict[item])
    trans.append(transaction)

In [5]:
Candidates = {}
accepted_itemsets = {}
supp_count_L = {}
min_support = 0.005
l = 1
Discarded = {l : []}
Candidates.update({l : [ [f] for f in items_dict.values()]})

In [6]:
def frequency_of_items(itemset, trans):
    count =0
    for i in range(len(trans)):
        if set(itemset).issubset(set(trans[i])):
            count += 1
    return count


def frequent_subsets(itemsets, transactions, min_support, prev_discarded):
    accepted = []
    supp = []
    new_discard = []
    
    k = len(prev_discarded.keys())
    
    for i in range(len(itemsets)):
        flag = pruning(itemsets[i], prev_discarded, k) 
        if flag == False:
            count = frequency_of_items(itemsets[i], trans)
            c = count/len(transactions)
            if c >= min_support:
                accepted.append(itemsets[i])
                supp.append(c)
            else:
                new_discard.append(itemsets[i])
    return accepted,supp,new_discard

def self_join(its):
    new_itemsets = []
    for i in range(len(its)):
        for j in range(i+1, len(its)):
            it_join = its[i] + its[j]
            if len(it_join) > 0:
                new_itemsets.append(it_join)
    return new_itemsets

def pruning(item, prev_discarded, k):
    flag = False
    if k > 0:
        for j in prev_discarded[k]:
            if set(j).issubset(item):
                flag = True
                break
    return flag
        

def table(items, support):
    print("Itemsets | Support")
    for k in range(len(items)):
        
        print("{} : {}".format(items[k], support[k]))
        print("\n\n")      

In [7]:
 ######### Generate L1 #############

f,supp,dis = frequent_subsets(Candidates[l], trans, min_support, Discarded)
Discarded.update({l : dis})
accepted_itemsets.update({l : f})
supp_count_L.update({l: supp})

In [8]:
 table(accepted_itemsets[1], supp_count_L[1])

Itemsets | Support
[1] : 0.008032536858159633



[2] : 0.03345195729537367



[8] : 0.017691916624300967



[10] : 0.05246568378240976



[11] : 0.033248601931875954



[12] : 0.026029486527707167



[13] : 0.08052872394509406



[14] : 0.11052364006100661



[16] : 0.06487036095577021



[17] : 0.05541433655312659



[18] : 0.027961362480935434



[19] : 0.013218098627351297



[20] : 0.008947635993899338



[21] : 0.0298932384341637



[22] : 0.07768174885612608



[23] : 0.015048296898830707



[25] : 0.010777834265378749



[26] : 0.023284189120488054



[27] : 0.0056939501779359435



[28] : 0.021047280122013217



[29] : 0.04290798169801729



[30] : 0.04961870869344179



[31] : 0.009049313675648195



[32] : 0.08276563294356888



[33] : 0.005083884087442806



[34] : 0.011387900355871887



[36] : 0.05805795627859685



[37] : 0.010269445856634469



[41] : 0.03965429588205389



[42] : 0.05327910523640061



[43] : 0.005083884087442806



[45] : 0.005795627859684799



[46] :

In [ ]:
k = l + 1
flag = False
while (flag == False):
    Candidates.update({k : self_join(accepted_itemsets[k-1])})
    new_itemsets,support_count,new_discarded = frequent_subsets(Candidates[k], trans, min_support, Discarded)
    Discarded.update({k : new_discarded})
    accepted_itemsets.update({k : new_itemsets})
    supp_count_L.update({ k : support_count})
    print("levels ", k)
    if len(accepted_itemsets[k]) ==0:
        flag = True
    k +=1